In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt

In [2]:
data_df = pd.read_excel('model_train_data_whole_1103.xlsx')

In [3]:
rating_list = ['AAApi','AAA', 'AAA-','AA+', 'AA','AA-','A+','A','A-','BBB+','BBB','BBB-','BB+','BB','BB-','B+','B','B-','CCC','CC','C']
macro_rating_list = [0,0,0,1,1,1,2,2,2,3,3,3,3,3,3,3,3,3,3,3,3]
rating_list_df = {'ratings': rating_list,'macro_ratings':macro_rating_list}
rating_list_df = pd.DataFrame(rating_list_df)
rating_dict = rating_list_df.reset_index().set_index(['ratings'])['macro_ratings'].to_dict()
data_df['sample ID'] = data_df.index
data_df['macro ratings number'] = data_df['信用评级'].apply(lambda x: rating_dict[x] )

In [4]:
data_df = data_df.drop(columns = ['Unnamed: 0','full name','short name','wind code','最新报告期','信用评级发布日期', '信用评级', '信用评级与报告差距天数','freeCashFlowOperatingCashFlowRatio','日期差距最小值','ratings number'])

In [5]:
data_df0 = data_df[data_df['macro ratings number']==0]
data_df1 = data_df[data_df['macro ratings number']==1]
data_df2 = data_df[data_df['macro ratings number']==2]
data_df3 = data_df[data_df['macro ratings number']==3]
data_df0_sample = data_df0.sample(200)
data_df1_sample = data_df1.sample(300)
data_df2_sample = data_df2.sample(200)
data_df3_sample = data_df3
data_df_sample = pd.concat([data_df0_sample,data_df1_sample,data_df2_sample,data_df3_sample])
data_df_sample.sample(frac = 1)

test_set_df = data_df_sample.sample(80)
test_set_df['is test data']=True
test_set_df = test_set_df[['is test data','sample ID']]
data_df = pd.merge(data_df,test_set_df,on = ['sample ID'],how = 'left')
data_df[data_df['is test data']==True]

train_data_df = data_df[data_df['is test data']!=True]
test_data_df = data_df[data_df['is test data']==True]
train_data_df = train_data_df.drop(columns = ['sample ID','is test data'])
test_data_df = test_data_df.drop(columns = ['sample ID','is test data'])

In [6]:
data_df0 = train_data_df[train_data_df['macro ratings number']==0]
data_df1 = train_data_df[train_data_df['macro ratings number']==1]
data_df2 = train_data_df[train_data_df['macro ratings number']==2]
data_df3 = train_data_df[train_data_df['macro ratings number']==3]


def prepare_training_samples_df(data_df0,data_df1,data_df2,data_df3):

    data_df0_sample = data_df0.sample(200)
    data_df1_sample = data_df1.sample(300)
    data_df2_sample = data_df2.sample(200)
    data_df3_sample = data_df3
    data_df_sample = pd.concat([data_df0_sample,data_df1_sample,data_df2_sample,data_df3_sample])
    data_df_sample.sample(frac = 1)
    col_names = data_df_sample.columns.values.tolist()
    columns = {}
    for idx, n in enumerate(col_names):
        if idx == len(col_names)-1:
            columns[n] = "y"
        else:
            columns[n] = "x%d"%idx
    data_df_sample = data_df_sample.rename(columns=columns)

    return data_df_sample




def prepare_testing_samples_df(test_df):
    
    col_names = test_df.columns.values.tolist()
    columns = {}
    for idx, n in enumerate(col_names):
        if idx == len(col_names)-1:
            columns[n] = "y"
        else:
            columns[n] = "x%d"%idx
    test_data_sample_df = test_df.rename(columns=columns)

    return test_data_sample_df

In [7]:
data_df_sample1 = prepare_training_samples_df(data_df0,data_df1,data_df2,data_df3)
data_df_sample2 = prepare_training_samples_df(data_df0,data_df1,data_df2,data_df3)
data_df_sample3 = prepare_training_samples_df(data_df0,data_df1,data_df2,data_df3)
data_df_sample4 = prepare_training_samples_df(data_df0,data_df1,data_df2,data_df3)
data_df_sample5 = prepare_training_samples_df(data_df0,data_df1,data_df2,data_df3)
data_df_sample1.to_csv('agency_data_1_train.csv', index=True, header=True)
data_df_sample2.to_csv('agency_data_2_train.csv', index=True, header=True)
data_df_sample3.to_csv('agency_data_3_train.csv', index=True, header=True)
data_df_sample4.to_csv('agency_data_4_train.csv', index=True, header=True)
data_df_sample5.to_csv('agency_data_5_train.csv', index=True, header=True)



test_data_sample_df = prepare_testing_samples_df(test_data_df)


test_data_sample_df.to_csv('test_data_1.csv', index=True, header=True)

In [8]:
!pipeline init --ip 127.0.0.1 --port 9380

Pipeline configuration succeeded.


In [9]:
from pipeline.backend.pipeline import PipeLine

In [10]:
dense_data_guest1 = {"name": "agency_guest1", "namespace": f"experiment"}
dense_data_guest2 = {"name": "agency_guest2", "namespace": f"experiment"}
dense_data_guest3 = {"name": "agency_guest3", "namespace": f"experiment"}
dense_data_guest4 = {"name": "agency_guest4", "namespace": f"experiment"}


dense_data_host = {"name": "agency_host", "namespace": f"experiment"}


test_data_guest1 = {"name": "agency_guest1", "namespace": f"test"}
test_data_guest2 = {"name": "agency_guest2", "namespace": f"test"}
test_data_guest3 = {"name": "agency_guest3", "namespace": f"test"}
test_data_guest4 = {"name": "agency_guest4", "namespace": f"test"}

test_data_host = {"name": "agency_host", "namespace": f"test"}

In [11]:
import os
pipeline_upload = PipeLine().set_initiator(role='guest', party_id=9999).set_roles(guest=9999)
partition = 1

data_base = "/data/projects/fate/JupyterLabRoot/"
pipeline_upload.add_upload_data(file=os.path.join(data_base, "agency_data_1_train.csv"),
                                table_name=dense_data_guest1["name"],             # table name
                                namespace=dense_data_guest1["namespace"],         # namespace
                                head=1, partition=partition)               # data info

pipeline_upload.add_upload_data(file=os.path.join(data_base, "agency_data_2_train.csv"),
                                table_name=dense_data_host["name"],
                                namespace=dense_data_host["namespace"],
                                head=1, partition=partition)

pipeline_upload.add_upload_data(file=os.path.join(data_base, "agency_data_3_train.csv"),
                                table_name=dense_data_guest2["name"],             # table name
                                namespace=dense_data_guest2["namespace"],         # namespace
                                head=1, partition=partition)  


pipeline_upload.add_upload_data(file=os.path.join(data_base, "agency_data_4_train.csv"),
                                table_name=dense_data_guest3["name"],             # table name
                                namespace=dense_data_guest3["namespace"],         # namespace
                                head=1, partition=partition)  


pipeline_upload.add_upload_data(file=os.path.join(data_base, "agency_data_5_train.csv"),
                                table_name=dense_data_guest4["name"],             # table name
                                namespace=dense_data_guest4["namespace"],         # namespace
                                head=1, partition=partition)  



pipeline_upload.add_upload_data(file=os.path.join(data_base, "test_data_1.csv"),
                                table_name=test_data_guest1["name"],
                                namespace=test_data_guest1["namespace"],
                                head=1, partition=partition)

pipeline_upload.add_upload_data(file=os.path.join(data_base, "test_data_1.csv"),
                                table_name=test_data_guest2["name"],
                                namespace=test_data_guest2["namespace"],
                                head=1, partition=partition)

pipeline_upload.add_upload_data(file=os.path.join(data_base, "test_data_1.csv"),
                                table_name=test_data_guest3["name"],
                                namespace=test_data_guest3["namespace"],
                                head=1, partition=partition)

pipeline_upload.add_upload_data(file=os.path.join(data_base, "test_data_1.csv"),
                                table_name=test_data_guest4["name"],
                                namespace=test_data_guest4["namespace"],
                                head=1, partition=partition)

pipeline_upload.add_upload_data(file=os.path.join(data_base, "test_data_1.csv"),
                                table_name=test_data_host["name"],
                                namespace=test_data_host["namespace"],
                                head=1, partition=partition)

In [12]:
pipeline_upload.upload(drop=1)

 UPLOADING:||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||100.00%


2022-11-11 12:52:26.135 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:123 - Job id is 202211111252254063730

2022-11-11 12:52:26.144 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:144 - Job is still waiting, time elapse: 0:00:00
m2022-11-11 12:52:26.669 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:173 - 
2022-11-11 12:52:26.670 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:177 - Running component upload_0, time elapse: 0:00:00
2022-11-11 12:52:27.184 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:177 - Running component upload_0, time elapse: 0:00:01
2022-11-11 12:52:27.698 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:177 - Running component upload_0, time elapse: 0:00:01
2022-11-11 12:52:28.215 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:177 - Running component upload_0, time elapse: 0:00:02
2022-11-11 12:52:29.800 | INFO   

 UPLOADING:||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||100.00%


2022-11-11 12:52:30.414 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:123 - Job id is 202211111252298085370

2022-11-11 12:52:30.423 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:144 - Job is still waiting, time elapse: 0:00:00
2022-11-11 12:52:30.934 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:144 - Job is still waiting, time elapse: 0:00:00
2022-11-11 12:52:31.442 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:144 - Job is still waiting, time elapse: 0:00:01
2022-11-11 12:52:31.952 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:144 - Job is still waiting, time elapse: 0:00:01
2022-11-11 12:52:32.460 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:144 - Job is still waiting, time elapse: 0:00:02
m2022-11-11 12:52:32.973 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:173 - 
2022-11-11 12:52:32.975 | INFO     | pipeline.utils.invok

 UPLOADING:||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||100.00%


...

2022-11-11 12:53:09.435 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:123 - Job id is 202211111253088241880

2022-11-11 12:53:09.445 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:144 - Job is still waiting, time elapse: 0:00:00
2022-11-11 12:53:09.954 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:144 - Job is still waiting, time elapse: 0:00:00
2022-11-11 12:53:10.462 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:144 - Job is still waiting, time elapse: 0:00:01
2022-11-11 12:53:10.973 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:144 - Job is still waiting, time elapse: 0:00:01
2022-11-11 12:53:11.482 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:144 - Job is still waiting, time elapse: 0:00:02
m2022-11-11 12:53:12.028 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:173 - 
2022-11-11 12:53:12.030 | INFO     | pipeline.utils.invok

 UPLOADING:||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||100.00%


2022-11-11 12:53:16.363 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:123 - Job id is 202211111253151268050

2022-11-11 12:53:16.372 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:144 - Job is still waiting, time elapse: 0:00:00
2022-11-11 12:53:16.883 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:144 - Job is still waiting, time elapse: 0:00:00
2022-11-11 12:53:17.391 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:144 - Job is still waiting, time elapse: 0:00:01
2022-11-11 12:53:17.904 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:144 - Job is still waiting, time elapse: 0:00:01
2022-11-11 12:53:18.413 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:144 - Job is still waiting, time elapse: 0:00:02
m2022-11-11 12:53:18.943 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:173 - 
2022-11-11 12:53:18.945 | INFO     | pipeline.utils.invok

 UPLOADING:||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||100.00%


2022-11-11 12:53:22.723 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:123 - Job id is 202211111253221394630

2022-11-11 12:53:22.732 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:144 - Job is still waiting, time elapse: 0:00:00
2022-11-11 12:53:23.240 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:144 - Job is still waiting, time elapse: 0:00:00
2022-11-11 12:53:23.751 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:144 - Job is still waiting, time elapse: 0:00:01
2022-11-11 12:53:24.238 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:144 - Job is still waiting, time elapse: 0:00:01
2022-11-11 12:53:24.746 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:144 - Job is still waiting, time elapse: 0:00:02
m2022-11-11 12:53:25.279 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:173 - 
2022-11-11 12:53:25.281 | INFO     | pipeline.utils.invok

In [13]:
from pipeline.backend.pipeline import PipeLine
from pipeline.component import DataTransform
from pipeline.component import Reader
from pipeline.component import HomoNN, Evaluation
from pipeline.interface import Data

In [14]:
reader_0 = Reader(name="reader_0")
# set guest parameter
reader_0.get_party_instance(role='guest', party_id=9999).component_param(
    table=dense_data_guest1)
# set host parameter
reader_0.get_party_instance(role='host', party_id=10000).component_param(
    table=dense_data_host)

reader_0.get_party_instance(role='guest', party_id=8888).component_param(
    table=dense_data_guest2)

reader_0.get_party_instance(role='guest', party_id=7777).component_param(
    table=dense_data_guest3)
reader_0.get_party_instance(role='guest', party_id=6666).component_param(
    table=dense_data_guest4)



In [15]:
data_transform_0 = DataTransform(name="data_transform_0")
# set guest parameter
data_transform_0.get_party_instance(role='guest', party_id=9999).component_param(
    with_label=True)

data_transform_0.get_party_instance(role='guest', party_id=8888).component_param(
    with_label=True)
data_transform_0.get_party_instance(role='guest', party_id=7777).component_param(
    with_label=True)

data_transform_0.get_party_instance(role='guest', party_id=6666).component_param(
    with_label=True)

data_transform_0.get_party_instance(role='host', party_id=[10000]).component_param(
    with_label=True)


In [16]:
homo_nn_0 = HomoNN(
    name="homo_nn_0", 
    max_iter=120, 
    batch_size = 32
    )

from tensorflow.keras.layers import Dense
homo_nn_0.add(Dense(32, activation="sigmoid", name="dense_1"))
homo_nn_0.add(Dense(16, activation="sigmoid", name="dense_2"))
homo_nn_0.add(Dense(4, activation="softmax", name="predictions"))


In [182]:
# class myCallback(tf.keras.callbacks.Callback):
#   def on_epoch_end(self, epoch, logs={}):
#     if(logs.get('accuracy') > 0.70):
#     # print("\nReached %2.2f%% accuracy, so stopping training!!" %(ACCURACY_THRESHOLD*100))   
#       self.model.stop_training = True
# callbacks = myCallback()

In [17]:
from tensorflow.keras import optimizers
from tensorflow import keras

homo_nn_0.compile(
    optimizer=keras.optimizers.RMSprop(), 
#     optimizer=optimizers.Adam(learning_rate=0.001), 
    metrics=["SparseCategoricalAccuracy"],
#     metrics=keras.metrics.SparseCategoricalAccuracy(),
    loss="sparse_categorical_crossentropy",

)





In [18]:
pipeline = PipeLine() \
        .set_initiator(role='guest', party_id=9999) \
        .set_roles(guest=[9999,8888,7777,6666], host=10000, arbiter=10000)
pipeline.add_component(reader_0)
pipeline.add_component(data_transform_0, data=Data(data=reader_0.output.data))
# pipeline.add_component(homo_nn_0, data=Data(train_data=data_transform_0.output.data))
pipeline.add_component(homo_nn_0, data=Data(train_data=data_transform_0.output.data))


pipeline.compile();

In [19]:
pipeline.fit()

2022-11-11 12:54:17.282 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:123 - Job id is 202211111254124300760

2022-11-11 12:54:17.293 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:144 - Job is still waiting, time elapse: 0:00:00
2022-11-11 12:54:17.801 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:144 - Job is still waiting, time elapse: 0:00:00
2022-11-11 12:54:18.309 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:144 - Job is still waiting, time elapse: 0:00:01
2022-11-11 12:54:18.822 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:144 - Job is still waiting, time elapse: 0:00:01
m2022-11-11 12:54:19.359 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:173 - 
2022-11-11 12:54:19.361 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:177 - Running component reader_0, time elapse: 0:00:02
2022-11-11 12:54:19.878 | INFO     | pipeline.utils

2022-11-11 12:54:49.900 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:177 - Running component homo_nn_0, time elapse: 0:00:32
2022-11-11 12:54:50.427 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:177 - Running component homo_nn_0, time elapse: 0:00:33
2022-11-11 12:54:50.971 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:177 - Running component homo_nn_0, time elapse: 0:00:33
2022-11-11 12:54:51.540 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:177 - Running component homo_nn_0, time elapse: 0:00:34
2022-11-11 12:54:52.086 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:177 - Running component homo_nn_0, time elapse: 0:00:34
2022-11-11 12:54:52.666 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:177 - Running component homo_nn_0, time elapse: 0:00:35
2022-11-11 12:54:53.240 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:177 - Running

2022-11-11 12:55:16.839 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:177 - Running component homo_nn_0, time elapse: 0:00:59
2022-11-11 12:55:17.360 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:177 - Running component homo_nn_0, time elapse: 0:01:00
2022-11-11 12:55:17.875 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:177 - Running component homo_nn_0, time elapse: 0:01:00
2022-11-11 12:55:18.415 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:177 - Running component homo_nn_0, time elapse: 0:01:01
2022-11-11 12:55:18.931 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:177 - Running component homo_nn_0, time elapse: 0:01:01
2022-11-11 12:55:19.460 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:177 - Running component homo_nn_0, time elapse: 0:01:02
2022-11-11 12:55:19.978 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:177 - Running

2022-11-11 12:55:37.022 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:177 - Running component homo_nn_0, time elapse: 0:01:19
2022-11-11 12:55:37.538 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:177 - Running component homo_nn_0, time elapse: 0:01:20
2022-11-11 12:55:38.054 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:177 - Running component homo_nn_0, time elapse: 0:01:20
2022-11-11 12:55:38.568 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:177 - Running component homo_nn_0, time elapse: 0:01:21
2022-11-11 12:55:39.085 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:177 - Running component homo_nn_0, time elapse: 0:01:21
2022-11-11 12:55:39.599 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:177 - Running component homo_nn_0, time elapse: 0:01:22
2022-11-11 12:55:40.115 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:177 - Running

2022-11-11 12:55:57.155 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:177 - Running component homo_nn_0, time elapse: 0:01:39
2022-11-11 12:55:57.669 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:177 - Running component homo_nn_0, time elapse: 0:01:40
2022-11-11 12:55:58.234 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:177 - Running component homo_nn_0, time elapse: 0:01:40
2022-11-11 12:55:58.747 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:177 - Running component homo_nn_0, time elapse: 0:01:41
2022-11-11 12:55:59.260 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:177 - Running component homo_nn_0, time elapse: 0:01:41
2022-11-11 12:55:59.775 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:177 - Running component homo_nn_0, time elapse: 0:01:42
2022-11-11 12:56:00.292 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:177 - Running

2022-11-11 12:56:20.500 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:177 - Running component homo_nn_0, time elapse: 0:02:03
2022-11-11 12:56:37.656 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:131 - Job is success!!! Job id is 202211111254124300760
2022-11-11 12:56:37.669 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:132 - Total time: 0:02:20


In [186]:
# pipeline.dump("pipeline_saved.pkl");

In [20]:
# pipeline = PipeLine.load_model_from_file('pipeline_saved.pkl')
pipeline.deploy_component([pipeline.data_transform_0, pipeline.homo_nn_0]);

In [188]:
# data_transform_1 = DataTransform(name="data_transform_1")
# # set guest parameter

# data_transform_1.get_party_instance(role='host', party_id=[10000]).component_param(
#     with_label=True)



In [21]:
# reader_1 = Reader(name="reader_1")
# reader_1.get_party_instance(role="host", party_id=10000).component_param(table=dense_data_guest4)
# # reader_1.get_party_instance(role="guest", party_id=9999).component_param(table=test_data_host)
evaluation_0 = Evaluation(name="evaluation_0", eval_type="multi")




reader_1 = Reader(name="reader_1")
# set guest parameter
reader_1.get_party_instance(role='guest', party_id=9999).component_param(
    table=test_data_guest1)
# set host parameter
reader_1.get_party_instance(role='host', party_id=10000).component_param(
    table=test_data_host)

reader_1.get_party_instance(role='guest', party_id=8888).component_param(
    table=test_data_guest2)

reader_1.get_party_instance(role='guest', party_id=7777).component_param(
    table=test_data_guest3)
reader_1.get_party_instance(role='guest', party_id=6666).component_param(
    table=test_data_guest4)

In [22]:
predict_pipeline = PipeLine()
predict_pipeline.add_component(reader_1)\
                .add_component(pipeline, 
                               data=Data(predict_input={pipeline.data_transform_0.input.data: reader_1.output.data}))\
                .add_component(evaluation_0, data=Data(data=pipeline.homo_nn_0.output.data));

In [23]:
predict_pipeline.predict()

2022-11-11 13:10:27.016 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:123 - Job id is 202211111310208044360

2022-11-11 13:10:27.029 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:144 - Job is still waiting, time elapse: 0:00:00
2022-11-11 13:10:27.540 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:144 - Job is still waiting, time elapse: 0:00:00
2022-11-11 13:10:28.049 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:144 - Job is still waiting, time elapse: 0:00:01
m2022-11-11 13:10:28.579 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:173 - 
2022-11-11 13:10:28.582 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:177 - Running component reader_1, time elapse: 0:00:01
2022-11-11 13:10:29.111 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:177 - Running component reader_1, time elapse: 0:00:02
2022-11-11 13:10:29.640 | INFO     | pipeline

2022-11-11 13:10:53.495 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:177 - Running component homo_nn_0, time elapse: 0:00:26
2022-11-11 13:10:53.991 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:177 - Running component homo_nn_0, time elapse: 0:00:26
2022-11-11 13:10:54.534 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:177 - Running component homo_nn_0, time elapse: 0:00:27
2022-11-11 13:10:55.095 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:177 - Running component homo_nn_0, time elapse: 0:00:28
2022-11-11 13:10:55.621 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:177 - Running component homo_nn_0, time elapse: 0:00:28
2022-11-11 13:10:56.188 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:177 - Running component homo_nn_0, time elapse: 0:00:29
2022-11-11 13:10:56.772 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:177 - Running

m2022-11-11 13:11:27.031 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:173 - 
2022-11-11 13:11:27.039 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:177 - Running component evaluation_0, time elapse: 0:01:00
2022-11-11 13:11:27.573 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:177 - Running component evaluation_0, time elapse: 0:01:00
2022-11-11 13:11:28.095 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:177 - Running component evaluation_0, time elapse: 0:01:01
2022-11-11 13:11:28.625 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:177 - Running component evaluation_0, time elapse: 0:01:01
2022-11-11 13:11:29.205 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:177 - Running component evaluation_0, time elapse: 0:01:02
2022-11-11 13:11:29.751 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:177 - Running component evaluation_0, time ela